# Notebook Supported Desaster Recovery - Synching External Locations and Credentials
---
Parameters:
* source workspace uri
* destination workspace uri


Clientid, clientsecret and tenantid are read from a keyvault

Then we need a mapping table credmap for credname, targetaccessconnectorid and
locmap for external location and url. The bronzeextloc will be an external location, which will be replicated by provider. That is why the location will not be updated to the new one.


In [0]:
dbutils.widgets.text("source_workspace", "", "Source Workspace")
dbutils.widgets.text("destination_workspace", "", "Destination Workspace")

In [0]:
source_workspace = dbutils.widgets.get("source_workspace")
destination_workspace = dbutils.widgets.get("destination_workspace")


In [0]:
clientid = dbutils.secrets.get('kvbacked', 'clientid')
clientsecret = dbutils.secrets.get('kvbacked', 'clientsecret')
tenantid = dbutils.secrets.get('kvbacked', 'tenantid')

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service import catalog


In [0]:
# variables
# this is a dictionary of credential names and how they map to the access connector ids
credmap = {
    "devcreds": "/subscriptions/2865292f-c1ed-4867-a66d-337eae300421/resourceGroups/rg-wcus-adb3600318-dev/providers/Microsoft.Databricks/accessConnectors/adbac-westcentralus-adb3600318-dev"
}

locmap = {
    "bronzextlocdev" : "abfss://bronze@dlg2devwestcentralusetms.dfs.core.windows.net/",
    "catextlocdev" : "abfss://fsms@dlg2metastoredevwestlp6m.dfs.core.windows.net/"
}

In [0]:
# create connections to source and destination workspace
sourceWs = WorkspaceClient(azure_client_id=clientid, azure_client_secret=clientsecret, azure_tenant_id=tenantid, host=source_workspace)
destWs = WorkspaceClient(azure_client_id=clientid, azure_client_secret=clientsecret, azure_tenant_id=tenantid, host=destination_workspace)

In [0]:
# get the source and destination credentials
source_creds = [x for x in sourceWs.storage_credentials.list()]
dest_creds = [x for x in destWs.storage_credentials.list()]
# compare the two lists and find, which creds are not existing in destination
# also filter out the credentials named adbws_* because these are for the workspaces and UC
source_cred_names = [x.name for x in source_creds if not x.name.startswith("adbws_")]
target_cred_names = [x.name for x in dest_creds if not x.name.startswith("adbws_")]
cred_diff = list(set(source_cred_names) - set(target_cred_names))
creds_to_create = [x for x in source_creds if x.name in cred_diff]

In [0]:
# main loop to create the credentials
for cred in creds_to_create:
    cred_name = cred.name
    cred_read_only = cred.read_only
    cred_comment = cred.comment

    cred_request = catalog.AzureManagedIdentityRequest(access_connector_id=credmap[cred_name])
    destWs.storage_credentials.create(
        azure_managed_identity=cred_request, 
        name=cred_name, 
        read_only=cred_read_only,
        comment=cred_comment
        )
    print(f"Created credential {cred_name}")
print('Credentials created ')

In [0]:
# prepare external locations
# first read source and destination external locations
source_extlocs = [x for x in sourceWs.external_locations.list()]
dest_extlocs = [x for x in destWs.external_locations.list()]

# extract the names and leave out all location starting with adbws_*
source_extloc_names = [x.name for x in source_extlocs if not x.name.startswith("adbws_")]
dest_extloc_names = [x.name for x in dest_extlocs if not x.name.startswith("adbws_")]

# find the difference of source and dest extlocs
extloc_diff = list(set(source_extloc_names) - set(dest_extloc_names))
extlocs_to_create = [x for x in source_extlocs if x.name in extloc_diff]

In [0]:
# main loop to create external locations
for loc in extlocs_to_create:
    loc_name = loc.name
    loc_cred_name = loc.credential_name
    loc_commnet = loc.comment
    loc_read_only = loc.read_only
    loc_url = loc.url if loc.name == 'bronzextlocdev' else locmap[loc.name]

    print(f"Creating external location {loc_name} with credential {loc_cred_name} and comment {loc_commnet} and url  {loc_url}")

    destWs.external_locations.create(
        name=loc_name,
        credential_name=loc_cred_name,
        comment=loc_commnet,
        read_only=loc_read_only,
        url=loc_url
    )
        
print("External locations created")
